In [1]:
## 데이터 분석 라이브러리
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import csv
import re
import time
import random
from tqdm.notebook import tqdm

## Selenium 관련 환경설정
from selenium import webdriver as wb
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys
# from selenium.webdriver.common.action_chains import ActionChains
from selenium.common.exceptions import NoSuchElementException, ElementNotInteractableException
from selenium.webdriver.common.desired_capabilities import DesiredCapabilities
caps = DesiredCapabilities.CHROME
caps["pageLoadStrategy"] = "none"

## bs 라이브러리
from bs4 import BeautifulSoup
import requests

# 페이지 로드 간소화 라이브러리
from selenium.webdriver.support.ui import WebDriverWait as wait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC

# webdriver options
options = wb.ChromeOptions()
options.add_argument('--headless')
options.add_argument('--no-sandbox')
# options.add_argument('disable-dev-shm-usage')

# 이미지 x
options.add_argument('--disable-images')
options.add_experimental_option("prefs", {'profile.managed_default_content_settings.images': 2})
options.add_argument('--blink-settings=imagesEnabled=false')

In [2]:
def get_playlist_links(url, playlist_nth_child):
    # Chrome Driver 초기화 (ChromeDriverManager를 사용)
    driver = wb.Chrome(options=options)
    # YouTube 플레이리스트 URL로 이동
    driver.get(url)
    ## 인플루언서 명 가져오기기
    # influ = driver.find_element(By.CSS_SELECTOR, "#text").text
    influ = wait(driver, 10).until(EC.presence_of_element_located((By.CSS_SELECTOR, "#text"))).text
    # 특정 플레이리스트 선택하기 (예: 7번째 플레이리스트)
    btn = driver.find_element(By.CSS_SELECTOR, f"#items > ytd-grid-playlist-renderer:nth-child({playlist_nth_child}) > ytd-playlist-thumbnail")
    btn.click()
    time.sleep(2)
    # 'items' 영역을 찾아 스크롤 다운
    element = driver.find_element(By.CSS_SELECTOR, "#container")
    driver.execute_script("arguments[0].scrollTop = arguments[0].scrollHeight", element)
    # 영상의 링크 및 이미지들 가져오기
    links = driver.find_elements(By.CSS_SELECTOR, '#wc-endpoint')
    for link in links:
        link_list.append(link.get_attribute('href')[:43])
    # driver.quit()
    return link_list

In [3]:
# 더보기 체크 함수
def try_click_element(driver, css_selector):
    try:
        button = driver.find_element(By.CSS_SELECTOR, css_selector)
        button.click()
    except (NoSuchElementException, ElementNotInteractableException):
        pass

In [4]:
def extract_video_details(driver):
    # 설명란의 '더보기' 확인 및 클릭
    try_click_element(driver, "#expand")
    # 동영상 제목 가져오기
    head = driver.find_element(By.CSS_SELECTOR, '#title > h1 > yt-formatted-string').text
    # 설명 태그 개수 확인 및 값 가져오기
    explain_count = driver.find_elements(By.CSS_SELECTOR, "#info > a")
    explain_tag = ''
    for i in explain_count:
        explain_tag += i.text + ' '
    # 설명란 값 가져오기
    explain = driver.find_element(By.CSS_SELECTOR, "#description-inline-expander > yt-attributed-string > span > span:nth-child(1)").text
    # 스크롤 다운하여 댓글 섹션 이동
    driver.execute_script("window.scrollTo(0, 500)")
    time.sleep(2)
    last_height = driver.execute_script("return document.documentElement.scrollHeight")
    while True:
        driver.execute_script("window.scrollTo(0, document.documentElement.scrollHeight);")
        time.sleep(1.5)

        new_height = driver.execute_script("return document.documentElement.scrollHeight")
        if new_height == last_height:
            break
        last_height = new_height
    time.sleep(1.5)
    # 정규 표현식을 사용하여 [시간] 가수 - 노래제목 형식에 맞는 문자열 찾기
    pattern = r"[\[\d:]*\s*]?([^]]+) - ([^\n]+)"
    cleaned_explain = (explain_tag + explain).strip().replace('\n', '').replace('\nby', '')
    # 댓글의 가수와 곡명 가져오기
    html_source = driver.page_source
    soup = BeautifulSoup(html_source, 'html.parser')
    comment_list = soup.select("#content-text")
    
    # 리스트에 추가
    for comment in comment_list:
        comment_text = comment.get_text()
        tracks = re.findall(pattern, comment_text)
        if tracks:
            for track in tracks:
                # cleaned_explain이 #으로 시작하는지 확인
                if cleaned_explain.startswith('#'):
                    # 정규 표현식 매치에서 가수 이름과 곡명 추출
                    explain_list.append(cleaned_explain)
                    head_list.append(head)
                    artist, title = track
                    artists_list.append(artist.strip())
                    titles_list.append(title.strip())
                    img_list.append(f"https://img.youtube.com/vi/{driver.current_url.split('v=')[1]}/maxresdefault.jpg")
            # 패턴과 일치하는 댓글을 찾으면 루프 종료
            print(len(explain_list), len(head_list), len(img_list),len(artists_list),len(titles_list))
            break
    return

In [5]:
# 사용 예시
url = 'https://www.youtube.com/@micasfu/playlists'
# 가져오고 싶은 플레이리스트의 순서
playlist_nth_child = 7
## 큐레이션 정보를 담을 리스트 생성
head_list= []
influ_list = []
explain_list = []
artists_list = []
titles_list = []
link_list = []
img_list = []
link_list = get_playlist_links(url, playlist_nth_child)
driver = wb.Chrome(options=options)
for link in tqdm(link_list):
    driver.get(link)
    time.sleep(2)
    extract_video_details(driver)
driver.quit()

  0%|          | 0/173 [00:00<?, ?it/s]

28 28 28 28 28
55 55 55 55 55
55 55 55 55 55
80 80 80 80 80
105 105 105 105 105
134 134 134 134 134
165 165 165 165 165
197 197 197 197 197
224 224 224 224 224
256 256 256 256 256
257 257 257 257 257
284 284 284 284 284
284 284 284 284 284
312 312 312 312 312
337 337 337 337 337
364 364 364 364 364
397 397 397 397 397
431 431 431 431 431
464 464 464 464 464
494 494 494 494 494
521 521 521 521 521
550 550 550 550 550
575 575 575 575 575
602 602 602 602 602
630 630 630 630 630
654 654 654 654 654
679 679 679 679 679
679 679 679 679 679
690 690 690 690 690
713 713 713 713 713
713 713 713 713 713
713 713 713 713 713
713 713 713 713 713
742 742 742 742 742
771 771 771 771 771
794 794 794 794 794
821 821 821 821 821
849 849 849 849 849
879 879 879 879 879
910 910 910 910 910
910 910 910 910 910
934 934 934 934 934
960 960 960 960 960
987 987 987 987 987
1011 1011 1011 1011 1011
1011 1011 1011 1011 1011
1038 1038 1038 1038 1038
1065 1065 1065 1065 1065
1066 1066 1066 1066 1066
1087 1087 1087 

In [190]:
print("이미지리스트\n",img_list)
print("제목리스트\n",head_list)
print("설명리스트\n",explain_list)
print("가수리스트\n",artists_list)
print("곡명리스트\n",titles_list)

이미지리스트
 ['https://img.youtube.com/vi/Cnh7exHmJ_U/maxresdefault.jpg', 'https://img.youtube.com/vi/Cnh7exHmJ_U/maxresdefault.jpg', 'https://img.youtube.com/vi/Cnh7exHmJ_U/maxresdefault.jpg', 'https://img.youtube.com/vi/Cnh7exHmJ_U/maxresdefault.jpg', 'https://img.youtube.com/vi/Cnh7exHmJ_U/maxresdefault.jpg', 'https://img.youtube.com/vi/Cnh7exHmJ_U/maxresdefault.jpg', 'https://img.youtube.com/vi/Cnh7exHmJ_U/maxresdefault.jpg', 'https://img.youtube.com/vi/Cnh7exHmJ_U/maxresdefault.jpg', 'https://img.youtube.com/vi/Cnh7exHmJ_U/maxresdefault.jpg', 'https://img.youtube.com/vi/Cnh7exHmJ_U/maxresdefault.jpg', 'https://img.youtube.com/vi/Cnh7exHmJ_U/maxresdefault.jpg', 'https://img.youtube.com/vi/Cnh7exHmJ_U/maxresdefault.jpg', 'https://img.youtube.com/vi/Cnh7exHmJ_U/maxresdefault.jpg', 'https://img.youtube.com/vi/Cnh7exHmJ_U/maxresdefault.jpg', 'https://img.youtube.com/vi/Cnh7exHmJ_U/maxresdefault.jpg', 'https://img.youtube.com/vi/Cnh7exHmJ_U/maxresdefault.jpg', 'https://img.youtube.com/vi/Cnh

In [17]:
dic = {'이미지' : img_list, '제목' : head_list, '설명':explain_list,'가수':artists_list,'곡명':titles_list}
data = pd.DataFrame(dic)
data['제목'] = data['제목'].str.replace('[playlist] ', '', regex=False)
data.to_csv("influ_data.csv", index=False, encoding="utf-8-sig")
data

,이미지,제목,설명,가수,곡명
0,https://img.youtube.com/vi/Cnh7exHmJ_U/maxresd...,사랑이 머문 자리에,#팝송추천 #플레이리스트 #노래추천 우리가 함께했던 수많은 공간에는 아직도 그때의 ...,Stephen Sanchez,Until I Found You
1,https://img.youtube.com/vi/Cnh7exHmJ_U/maxresd...,사랑이 머문 자리에,#팝송추천 #플레이리스트 #노래추천 우리가 함께했던 수많은 공간에는 아직도 그때의 ...,"MAX, keshi",IT'S YOU
2,https://img.youtube.com/vi/Cnh7exHmJ_U/maxresd...,사랑이 머문 자리에,#팝송추천 #플레이리스트 #노래추천 우리가 함께했던 수많은 공간에는 아직도 그때의 ...,keshi,UNDERSTAND
3,https://img.youtube.com/vi/Cnh7exHmJ_U/maxresd...,사랑이 머문 자리에,#팝송추천 #플레이리스트 #노래추천 우리가 함께했던 수많은 공간에는 아직도 그때의 ...,Arash Buana,stars
4,https://img.youtube.com/vi/Cnh7exHmJ_U/maxresd...,사랑이 머문 자리에,#팝송추천 #플레이리스트 #노래추천 우리가 함께했던 수많은 공간에는 아직도 그때의 ...,"Jesse Barrera, Nieman, Melissa Polinar",Tangled Up (Reimagined)
...,...,...,...,...,...
2296,https://img.youtube.com/vi/1QEUJ5d7WO8/maxresd...,나만 알고 싶은 새벽감성 사클 노래 모음,#노래추천 #음악추천 #사운드클라우드 구독자 1000명! 다들 너무 고마워요!그래서...,Bas Bao,"넌 없고, 난 머물고"
2297,https://img.youtube.com/vi/1QEUJ5d7WO8/maxresd...,나만 알고 싶은 새벽감성 사클 노래 모음,#노래추천 #음악추천 #사운드클라우드 구독자 1000명! 다들 너무 고마워요!그래서...,D'sperado,"O/W (with hatts, HYE SUNG)"
2298,https://img.youtube.com/vi/1QEUJ5d7WO8/maxresd...,나만 알고 싶은 새벽감성 사클 노래 모음,#노래추천 #음악추천 #사운드클라우드 구독자 1000명! 다들 너무 고마워요!그래서...,DAWN,불면증 (白夜) (feat. YAYYOUNG) (prod. by N O V E L)
2299,https://img.youtube.com/vi/1QEUJ5d7WO8/maxresd...,나만 알고 싶은 새벽감성 사클 노래 모음,#노래추천 #음악추천 #사운드클라우드 구독자 1000명! 다들 너무 고마워요!그래서...,Mabinc,혼잣말 (feat. Ripley)


In [18]:
dic = {'이미지' : img_list, '제목' : head_list, '설명':explain_list,'가수':artists_list,'곡명':titles_list}
data = pd.DataFrame(dic)
data.index.name = '번호'
# '[playlist] ' 부분을 제거 (정규 표현식 사용하지 않음)
data['제목'] = data['제목'].str.replace('[playlist] ', '', regex=False)
data.to_csv("influ_data_index.csv", index=False, encoding="utf-8-sig")
data

,이미지,제목,설명,가수,곡명
번호,,,,,
0,https://img.youtube.com/vi/Cnh7exHmJ_U/maxresd...,사랑이 머문 자리에,#팝송추천 #플레이리스트 #노래추천 우리가 함께했던 수많은 공간에는 아직도 그때의 ...,Stephen Sanchez,Until I Found You
1,https://img.youtube.com/vi/Cnh7exHmJ_U/maxresd...,사랑이 머문 자리에,#팝송추천 #플레이리스트 #노래추천 우리가 함께했던 수많은 공간에는 아직도 그때의 ...,"MAX, keshi",IT'S YOU
2,https://img.youtube.com/vi/Cnh7exHmJ_U/maxresd...,사랑이 머문 자리에,#팝송추천 #플레이리스트 #노래추천 우리가 함께했던 수많은 공간에는 아직도 그때의 ...,keshi,UNDERSTAND
3,https://img.youtube.com/vi/Cnh7exHmJ_U/maxresd...,사랑이 머문 자리에,#팝송추천 #플레이리스트 #노래추천 우리가 함께했던 수많은 공간에는 아직도 그때의 ...,Arash Buana,stars
4,https://img.youtube.com/vi/Cnh7exHmJ_U/maxresd...,사랑이 머문 자리에,#팝송추천 #플레이리스트 #노래추천 우리가 함께했던 수많은 공간에는 아직도 그때의 ...,"Jesse Barrera, Nieman, Melissa Polinar",Tangled Up (Reimagined)
...,...,...,...,...,...
2296,https://img.youtube.com/vi/1QEUJ5d7WO8/maxresd...,나만 알고 싶은 새벽감성 사클 노래 모음,#노래추천 #음악추천 #사운드클라우드 구독자 1000명! 다들 너무 고마워요!그래서...,Bas Bao,"넌 없고, 난 머물고"
2297,https://img.youtube.com/vi/1QEUJ5d7WO8/maxresd...,나만 알고 싶은 새벽감성 사클 노래 모음,#노래추천 #음악추천 #사운드클라우드 구독자 1000명! 다들 너무 고마워요!그래서...,D'sperado,"O/W (with hatts, HYE SUNG)"
2298,https://img.youtube.com/vi/1QEUJ5d7WO8/maxresd...,나만 알고 싶은 새벽감성 사클 노래 모음,#노래추천 #음악추천 #사운드클라우드 구독자 1000명! 다들 너무 고마워요!그래서...,DAWN,불면증 (白夜) (feat. YAYYOUNG) (prod. by N O V E L)
